In [7]:
import numpy as np
import random as rnd
import time as tm
from numpy import linalg as Lalg

# SUBMIT YOUR CODE AS A SINGLE PYTHON (.PY) FILE INSIDE A ZIP ARCHIVE
# THE NAME OF THE PYTHON FILE MUST BE SUBMIT.PY
# DO NOT INCLUDE PACKAGES LIKE SKLEARN, SCIPY, KERAS ETC IN YOUR CODE
# THE USE OF ANY MACHINE LEARNING LIBRARIES FOR WHATEVER REASON WILL RESULT IN A STRAIGHT ZERO
# THIS IS BECAUSE THESE PACKAGES CONTAIN SOLVERS WHICH MAKE THIS ASSIGNMENT TRIVIAL

# DO NOT CHANGE THE NAME OF THE METHOD "solver" BELOW. THIS ACTS AS THE MAIN METHOD AND
# WILL BE INVOKED BY THE EVALUATION SCRIPT. CHANGING THIS NAME WILL CAUSE EVALUATION FAILURES

# You may define any new functions, variables, classes here
# For example, functions to calculate next coordinate or step length

################################
# Non Editable Region Starting #
################################
def solver( X, y, timeout, spacing ):
	(n, d) = X.shape
	t = 0
	totTime = 0
	
	# w is the model vector and will get returned once timeout happens
	w = np.zeros( (d,) )
	tic = tm.perf_counter()
################################
#  Non Editable Region Ending  #
################################
    
	# You may reinitialize w to your liking here
	# You may also define new variables here e.g. step_length, mini-batch size etc
	R = 3
	alpha = 36
	l = 1/R
	#w_ = np.zeros((d,))
	myloss = 0
    
	def soft_thres(W, hyper):
		for i in range(0,len(W)):
			if (W[i] > hyper):
				W[i] = W[i]-hyper
			elif (W[i] < hyper):
				W[i] = W[i] + hyper
			else:
				W[i] = 0
		return W

################################
# Non Editable Region Starting #
################################
	while True:
		t = t + 1
		if t % spacing == 0:
			toc = tm.perf_counter()
			totTime = totTime + (toc - tic)
			if totTime > timeout:
				return (w, totTime) 
			else:
				tic = tm.perf_counter()
################################
#  Non Editable Region Ending  #
################################

		# Write all code to perform your method updates here within the infinite while loop
		# The infinite loop will terminate once timeout is reached
		# Do not try to bypass the timer check e.g. by using continue
		# It is very easy for us to detect such bypasses which will be strictly penalized
		
		# Please note that once timeout is reached, the code will simply return w
		# Thus, if you wish to return the average model (as is sometimes done for GD),
		# you need to make sure that w stores the average at all times
		# One way to do so is to define a "running" variable w_run
		# Make all GD updates to w_run e.g. w_run = w_run - step * delw
		# Then use a running average formula to update w
		# w = (w * (t-1) + w_run)/t
		# This way, w will always store the average and can be returned at any time
		# In this scheme, w plays the role of the "cumulative" variable in the course module optLib
		# w_run on the other hand, plays the role of the "theta" variable in the course module optLib
        
		l = 1/R
		diff = (X.dot(w)-y)
		w_ = w - l*(X.T).dot(diff)
        
        #Soft-Thresholding_call
		w_ = soft_thres(w_, l)
        
        #Hyperparameter Tuning step
		while (Lalg.norm(y-X.dot(w_),2)**2 > Lalg.norm(y-X.dot(w),2)**2 + (np.transpose(w_-w)).dot((( np.transpose(X) ).dot(X.dot(w)-y))) + (R/2)*(Lalg.norm((w_-w),2) ) ):
			R = alpha*R
			l = 1/R
        
		w = w_
        #This is the loss function
		myloss = Lalg.norm(y-X.dot(w_), 2)**2 + Lalg.norm(w_ ,1)
		#print (myloss)
        
	print (myloss)    
	return (w, totTime) # This return statement will never be reached 
    

In [8]:
# Find out how much loss is the learnt model incurring?
def getObjValue( X, y, wHat ):
	lassoLoss = np.linalg.norm( wHat, 1 ) + pow( np.linalg.norm( X.dot( wHat ) - y, 2 ), 2 )
	return lassoLoss

# Find out how far is the learnt model from the true one in terms of Euclidean distance
def getModelError( wHat, wAst ):
	return np.linalg.norm( wHat - wAst, 2 )

# Force the learnt model to become sparse and then see how well it approximates the true model
def getSupportError( wHat, wAst, k ):
	# Find the k coordinates where the true model has non-zero values
	idxAst = np.abs( wAst ).argsort()[::-1][:k]
	# Find the k coordinates with largest values (absolute terms) in the learnt model
	idxHat = np.abs( wHat ).argsort()[::-1][:k]
	
	# Set up indicator arrays to find the diff between the two
	# Could have used Python's set difference function here as well
	a = np.zeros_like( wAst )
	a[idxAst] = 1
	b = np.zeros_like( wAst )
	b[idxHat] = 1
	return np.linalg.norm( a - b, 1 )//2

# Z = np.loadtxt( "train" )
# wAst = np.loadtxt( "wAstTrain" )
Z = np.loadtxt( r"H:\CS 771\assn1\train" )
#print (Z.shape)
wAst = np.loadtxt( r"H:\CS 771\assn1\wAstTrain" )
k = 20

y = Z[:,0]
X = Z[:,1:]

# To avoid unlucky outcomes try running the code several times
numTrials = 5

# Try various timeouts - the timeouts are in seconds
timeouts = np.array( [0.1, 1, 2, 5] )

# Try checking for timeout every 10 iterations
spacing = 10

result = np.zeros( (len( timeouts ), 4) )

for i in range(0, len( timeouts )):
	to = timeouts[i]
	avgObj = 0
	avgDist = 0
	avgSupp = 0
	avgTime = 0
	for t in range( numTrials ):
		(w, totTime) = solver( X, y, to, spacing )
		avgObj = avgObj + getObjValue( X, y, w )
		avgDist = avgDist + getModelError( w, wAst )
		avgSupp = avgSupp + getSupportError( w, wAst, k )
		avgTime = avgTime + totTime
	result[i, 0] = avgObj/numTrials
	result[i, 1] = avgDist/numTrials
	result[i, 2] = avgSupp/numTrials
	result[i, 3] = avgTime/numTrials

np.savetxt( "result", result, fmt = "%.6f" )